# Overview
This code analyse data for emotional oddball task in children with severe mental health challenges. Overview of task, and population, and results from analysing a part of data can be found:

* 
* Electrophysiological correlates of fearful and sad distraction on target processing in adolescents with attention deficit-hyperactivity symptoms and affective disorders. 
* Singhal A, Shafer AT, Russell M, Gibson B, Wang L, Vohra S, Dolcos F. 
* Front Integr Neurosci. 2012 Dec 19;6:119. doi: 10.3389/fnint.2012.00119. PMID: 23267319; PMCID: PMC3525949.

## Task 
* Participants performed a modified version of the emotional oddball paradigm (Wang et al.,2005) which consisted of frequent stimuli serving as the baseline[scrambled pictures, 79% (465 trials)],infrequent distracters and odd ball targets, 21% (124 trials).Infrequent distracters consisted of sad and fearful pictures (13 trials each), neutral pictures (26 trials), and positive pictures (4 trials).The oddball targets(circles)were sub-grouped according to their preceding infrequent stimulus type [i.e.,target-after-sad (11 trials), target-after-fear (11 trials), target-after-target (24 trials), andtarget-after-neutral stimuli(22trials)].

* The participants’ task was to indicate whether the stimulus was a target or non-target by pressing a button. Participants were instructed to make a right hand button press any time they saw a target (circle) and a lef thand button press toall other stimuli (i.e., frequent scrambled and infrequent sad, fearful, neutral, and positive distracters). The inital , with targets and distractors. 

## Analysis 
This code extracts peaks and latency of ERP signals in a set of sites, whose electrode number and time window is specified in the code, and time series are given in 'C:\\QC\\'. 

As data is collected from children and adolescents with sever mental challenges, movement and noise of data was considerable. Subject's waveforms for AllDistractors
* AllDistractors = {'Fear','Neu','Sad','Standard'},

and AllTargets conditions  
* AllTargets = {'TargetafterFear','TargetafterNeu','TargetafterSad','TargetafterTarget'}

over Oz, O1, O2, T5, and T6 sites were first visually inpsected. Noisy Waveform were discarded from further analysis. The waveforms that survived this visual inspection are kept in "C:\good\" folder (sample for one subject is given).

Based on the survived participants' electrode/condition in the good data, peak and latency of ERP signals over the electrodes of interested were generated or discarded, based on the following association: 

- Good 'Oz' ==> generated 'Pz','Cz','PCz','POz','Oz'
- Good 'T5' ==> generated 'T5','PL','CL'
- Good 'T6' ==> generated 'T6','PR','CR'
- Good 'O1' ==> generated 'POL'
- Good 'O2' ==> generated 'POR'


- Good 'AllDistractors' ==> generated 'Fear','Neu','Sad','Standard'
- Good 'AllTargets'     ==> generated 'TargetafterFear','TargetafterNeu','TargetafterSad','TargetafterTarget'

Importing packages

In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from collections import namedtuple
from scipy import signal
import numpy as np

from os.path import exists
import sys
from numpy import NaN, Inf, arange, isscalar, asarray, array



Populating the interactive namespace from numpy and matplotlib


In [2]:
Result = namedtuple('Result', 'subject condition')
#Event = namedtuple('Event', 'timestamp value mean') # Old
Event = namedtuple('Event', 'timestamp value mean status') # New

Defining path, interval, and channels

In [3]:
my_path='C:\\QC\\'

good_path='C:\\Good\\'
#good_path='C:\\Good2\\'


start_time = -300   # ms
sample_interval = 4 # ms

Cz_channels = (257, 45, 132) #P3
PCz_channels = (90, 121, 80) #P3
Pz_channels = (101, 100, 129) #P3
POz_channels = (126, 118, 127) #P1 & P3
CL_channels = (59, 58, 65) #P3
CR_channels = (183, 195, 182) #P3
PL_channels = (86, 76, 85) #P3
PR_channels = (162, 171, 172) #P3
POL_channels = (108, 109, 117) #P1 & P3
POR_channels = (151, 140, 139) #add OL and OR channels P1 & P3

O1_channels = (116, 108,115) #P1 
O2_channels = (150, 151,159) #P1 
Oz_channels = (126,125,138) #P1 & P3?
T5_channels = (95, 96, 106) #P1 & P3?
T6_channels = (169, 170, 178) #P1 & P3?

#If Oz is good --> Central will be kept
#If O1 is good, POL is good
#If O2 is good, POR is good
#If T5 is good, PL and CL is good
#If T6 is good, PR and CR is good

# main = {'Oz':['Pz','Cz','PCz','POz'],'O1':['POL'],'O2':['POR'],'T5':['PL','CL'],'T6':['PR','CR']}
channel_association_P3 = {'Oz':['Pz','Cz','PCz'],'T5':['PL','CL'],'T6':['PR','CR']}
channel_association_Both = {'Oz':['POz','Oz'],'O1':['POL'],'O2':['POR'],'T5':['T5'],'T6':['T6']}
channel_association_LPP = {'Oz':['Pz'],'T5':['PL','T5'],'T6':['PR','T6']}

channel_association_P3All = {'Oz':['Pz','Cz','PCz','POz','Oz'],'T5':['T5','PL','CL'],'T6':['T6','PR','CR'],'O1':['POL'],'O2':['POR']}
channel_association_P1All = {'Oz':['POz','Oz'],'O1':['POL'],'O2':['POR'],'T5':['T5'],'T6':['T6']}


# desire_channel_P3 = [Cz_channels, Pz_channels, PCz_channels, CL_channels, CR_channels, PL_channels, PR_channels]
# desire_channel_P3name = ['Cz', 'Pz', 'PCz','CL', 'CR', 'PL', 'PR']

# desire_channel_bothP = [POz_channels, POL_channels, POR_channels, O1_channels, O2_channels, Oz_channels, T5_channels, T6_channels]
# desire_channel_bothPname = ['POz', 'POL','POR','O1','O2','Oz','T5','T6']

# desire_channel_LPP = [Pz_channels, PL_channels, PR_channels, T5_channels, T6_channels]
# desire_channel_LPPPname = ['Pz', 'PL','PR','T5','T6']


P3_start_time, P3_end_time = 250, 550 # Peak 
P1_start_time, P1_end_time = 50, 200 # Peak
LPP_start_time, LPP_end_time = 650, 950 # mean 

In [4]:
def convert_to_channel(x):
    return x + 1

def time_conversion_function(start_time, sample_interval):
    def conversion(x):
        return start_time + x * sample_interval
    return conversion

In [5]:
def average_channel_values(result, channels):
    selected_channels = result.loc[:, channels]
    return selected_channels.mean(axis=1)

In [8]:
def peakdet(v, delta, x = None):
#peakdetect.py 
#https://gist.github.com/endolith/250860#file-peakdetect-py
    """
    Converted from MATLAB script at http://billauer.co.il/peakdet.html
    Returns two arrays
    function [maxtab, mintab]=peakdet(v, delta, x)
    %PEAKDET Detect peaks in a vector
    %        [MAXTAB, MINTAB] = PEAKDET(V, DELTA) finds the local
    %        maxima and minima ("peaks") in the vector V.
    %        MAXTAB and MINTAB consists of two columns. Column 1
    %        contains indices in V, and column 2 the found values.
    %      
    %        With [MAXTAB, MINTAB] = PEAKDET(V, DELTA, X) the indices
    %        in MAXTAB and MINTAB are replaced with the corresponding
    %        X-values.
    %
    %        A point is considered a maximum peak if it has the maximal
    %        value, and was preceded (to the left) by a value lower by
    %        DELTA.
    
    % Eli Billauer, 3.4.05 (Explicitly not copyrighted).
    % This function is released to the public domain; Any use is allowed.
    """

    maxtab = []
    mintab = []
    if x is None:
        x = arange(len(v))
    v = asarray(v)
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    lookformax = True
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True
    return array(maxtab), array(mintab)


In [9]:
def find_maximum_and_mean_value_in_time_window(time_series, start_time_In, end_time_In):
#     from matplotlib.pyplot import plot, scatter, show
    
    ExtensionTimeLeft = 5 #ms
    ExtensionTimeRight = 5 #ms
    peak_res = .01 
    
    # Baseline noise levvel
    time_slice = time_series.loc[-300-1:0]
    baseline_std = time_slice.std()
    baseline_max = abs(time_slice).max()
    # print('Baseline std:',baseline_std,'& Baseilne Max:',baseline_max)
    # Ideas: check baseline_std, if it is too noisy: discard this case
    # check max_basesline, Peak should be above that
    # check the slop of the waveform, it should be more than 25, 
    # check peak amplitude, it should be more than thr

    
    time_slice = time_series.loc[start_time_In-1:end_time_In]
    start_time = time_slice.index[0]
    end_time = time_slice.index[-1]
#     print(start_time,end_time)
    maxtab, mintab = peakdet(time_slice,peak_res)
    if len(array(maxtab))>0:
        myindexmax = [int(x) for x in floor(array(maxtab)[:,0])]
#         scatter(time_slice.index[myindexmax], array(maxtab)[:,1], color='blue',marker='x')
        LeftMax=array([time_slice.index[myindexmax], array(maxtab)[:,1]])
    else: 
        LeftMax = np.array([(),()]) 
        
    time_slice_rev = time_series.loc[end_time:start_time:-1]
    maxtab_rev, mintab_rev = peakdet(time_slice_rev,peak_res)
    if len(array(maxtab_rev))>0:
        myindexmax_rev = [int(x) for x in floor(array(maxtab_rev)[:,0])]
#         scatter(time_slice_rev.index[myindexmax_rev], array(maxtab_rev)[:,1], color='cyan',marker='x')
        RightMax=array([time_slice_rev.index[myindexmax_rev], maxtab_rev[:,1]])
    else: 
        RightMax = np.array([(),()])  
        
#     print('888888888888888888888888888888888888888888888888888888888888')
#     print(time_slice)
#     print(time_slice_rev)
#     print('maxtab =',maxtab)
#     print('maxtab_rev =',maxtab_rev)

#     plot(time_slice)
    maxtabConc=np.concatenate((LeftMax,RightMax),axis=1)
#     print('LeftMax=',LeftMax)
#     print('RightMax=',RightMax)
#     print('maxtabConc=',maxtabConc)
    
    maxLocalInd = maxtabConc[1,:].argmax()
    maxLocalMax = maxtabConc[1,maxLocalInd];
    maxLocalIndOut = maxtabConc[0,maxLocalInd] 
#     print('maxLocalMax = ',maxLocalMax)
#     print('maxLocalInd = ',maxtabConc[0,maxLocalInd])

#     print('tmp:',maxtabConc[0,maxLocalInd])
#     print('tt:',time_slice.index[0])
    if maxLocalIndOut == start_time: # Peak at start point
        # Extend to the left and look for the first peak
        time_sliceExt_rev = time_series.loc[start_time:start_time-ExtensionTimeLeft:-1]  
#         plot(time_sliceExt_rev)
        maxtabExt_rev, mintabExt_rev = peakdet(time_sliceExt_rev,peak_res)
        if len(array(maxtabExt_rev))>0: # peak was found by extending to the left
            maxLocalMax = maxtabExt_rev[0,1]
            myindexmaxExt_rev = [int(x) for x in floor(array(maxtabExt_rev)[:,0])]
            maxLocalIndOut = time_sliceExt_rev.index[myindexmaxExt_rev[0]]
#             print('haha!! myindexmaxExt_rev=',myindexmaxExt_rev)
#             scatter(time_sliceExt_rev.index[myindexmaxExt_rev[0]], array(maxtabExt_rev)[0,1], color='purple',marker='o')
            Status = 'L'
        else: # peak was NOT found by extending to the left 
            # check again for the local max in the main area
            LeftMax = np.delete(LeftMax, 0, axis=1) # remove the first column which was left boundry max
            maxtabConc = np.concatenate((LeftMax,RightMax),axis=1)
            if len(array(maxtabConc[1,:]))>0:
                maxLocalInd = maxtabConc[1,:].argmax()
                maxLocalMax = maxtabConc[1,maxLocalInd];
                maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #             print('tmp,maxlocalMax=',maxLocalMax)
    #             print('boundry=',time_series.loc[end_time:end_time+sample_interval-1].mean())
                if  maxLocalIndOut == end_time:# Peak at end point:
                    # Extend to the right and look for peak
                    time_sliceExt = time_series.loc[end_time:end_time+ExtensionTimeRight]
    #                 plot(time_sliceExt)
                    maxtabExt, mintabExt = peakdet(time_sliceExt,peak_res)
                    if len(array(maxtabExt))>0:
                        maxLocalMax = maxtabExt[0,1]            
                        myindexmaxExt = [int(x) for x in floor(array(maxtabExt)[:,0])]
                        maxLocalIndOut = time_sliceExt.index[myindexmaxExt[0]]
    #                     scatter(time_sliceExt.index[myindexmaxExt], array(maxtabExt)[:,1], color='purple',marker='o')
                        Status = 'R'
                    else: 
                        # check again for the local max in the boundry
                        RightMax = np.delete(RightMax, 0, axis=1) # remove the first column which was right boundry max
    #                     print(RightMax)
                        maxtabConc=np.concatenate((LeftMax,RightMax),axis=1)
    #                     print(maxtabConc)
                        if len(array(maxtabConc[1,:]))>0:
                            maxLocalInd = maxtabConc[1,:].argmax()
                            maxLocalMax = maxtabConc[1,maxLocalInd];
                            maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #                         scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                            Status = 'G'
                        else:
                            #Nothing is found!
                            maxLocalMax = 999
                            maxLocalIndOut = 999
                            Status = 'NaN'            
                else: 
    #                 scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                    Status = 'G'
            else:
                #Nothing is found!
                maxLocalMax = 999
                maxLocalIndOut = 999
                Status = 'NaN'                                 
            
    elif maxLocalIndOut == end_time:# Peak at end point
        # Extend to the right and look for the first peak
        
        time_sliceExt = time_series.loc[end_time:end_time+ExtensionTimeRight]
#         plot(time_sliceExt)
        maxtabExt, mintabExt = peakdet(time_sliceExt,peak_res)
        if len(array(maxtabExt))>0: # peak was found by extending to the right
            maxLocalMax = maxtabExt[0,1]            
            myindexmaxExt = [int(x) for x in floor(array(maxtabExt)[:,0])]
            maxLocalIndOut = time_sliceExt.index[myindexmaxExt[0]]
#             print('haha!! myindexmaxExt=',myindexmaxExt)
#             scatter(time_sliceExt.index[myindexmaxExt[0]], array(maxtabExt)[0,1], color='purple',marker='o')
            Status = 'R'
        else: # peak was NOT found by extending to the right 
            # check again for the local max in the main area
            RightMax = np.delete(RightMax, 0, axis=1) # remove the first column which was right boundry max
            # check again for the local max in the boundry
            maxtabConc = np.concatenate((LeftMax,RightMax),axis=1)
#             print('RightMaxNew=',RightMax)

            if len(array(maxtabConc[1,:]))>0:
                maxLocalInd = maxtabConc[1,:].argmax()
                maxLocalMax = maxtabConc[1,maxLocalInd];
                maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #             print('tmp,maxlocalMax=',maxLocalMax)
    #             print('boundry=',time_series.loc[start_time:start_time+sample_interval-1].mean())
                if maxLocalIndOut == time_slice.index[0]:# Peak at start point:
                    # Extend to the left and look for the first peak
                    time_sliceExt_rev = time_series.loc[start_time:start_time-ExtensionTimeLeft:-1]   
    #                 plot(time_sliceExt_rev)
                    maxtabExt_rev, mintabExt_rev = peakdet(time_sliceExt_rev,peak_res)                
                    if len(array(maxtabExt_rev))>0:
                        maxLocalMax = maxtabExt_rev[0,1]            
                        myindexmaxExt = [int(x) for x in floor(array(maxtabExt_rev)[:,0])]
                        maxLocalIndOut = time_sliceExt_rev.index[myindexmaxExt[0]]
    #                     scatter(time_sliceExt_rev.index[myindexmaxExt], array(maxtabExt_rev)[:,1], color='purple',marker='o')
                        Status = 'L'
                    else:
                        # check again for the local max in the boundry
                        LeftMax = np.delete(LeftMax, 0, axis=1) # remove the first column which was left boundry max
    #                     print(LeftMax)
                        maxtabConc=np.concatenate((LeftMax,RightMax),axis=1)
    #                     print(maxtabConc)
                        if len(array(maxtabConc[1,:]))>0:
                            maxLocalInd = maxtabConc[1,:].argmax()
                            maxLocalMax = maxtabConc[1,maxLocalInd];
                            maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #                         scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                            Status = 'G'
                        else:
                            #Nothing is found!
                            maxLocalMax = 999
                            maxLocalIndOut = 999
                            Status = 'NaN'            
                else: 
    #                 scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                    Status = 'G'        
            else:
                #Nothing is found!
                maxLocalMax = 999
                maxLocalIndOut = 999
                Status = 'NaN'                
    else:
#         scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
        Status = 'G'
#     show()
#     print(time_series.loc[start_time:start_time+sample_interval-1])
#     print('1st sample =',time_series.loc[start_time:start_time+sample_interval-1].mean())
#     print('Overal Max =',maxLocalMax)
#     print(Status)
    
       
    #return Event(timestamp=time_slice.idxmax(), value=time_slice.max(), mean=time_slice.mean()) # old
    return Event(timestamp=int(maxLocalIndOut), value=maxLocalMax, mean=time_slice.mean(), status=Status) # new

In [28]:
myConditionDict = dict(
     AllDistractors = {'Fear','Neu','Sad','Standard'},
     AllTargets = {'TargetafterFear','TargetafterNeu','TargetafterSad','TargetafterTarget'}
)

result_map = {}
Waveform_All = {}
for allCond in myConditionDict.keys():
    input_file_pattern = good_path+'*_'+allCond+'_*.png'
    for filename in glob(input_file_pattern):
#         print(filename)
        mylocation = filename.find(allCond)
        last_slash=(filename.rfind('\\'))
        participantid = filename[last_slash+1:mylocation-1]
        goodsite = filename[mylocation+len(allCond)+1:mylocation+len(allCond)+3]
        filenameBase = my_path + participantid + '_'
#         print('filenameBase: '+ filenameBase)
        if goodsite in channel_association_P1All.keys():
#             print('good')
        
            for goodchannels in channel_association_P1All[goodsite]:
#                 print('goodchannels: '+ goodchannels)
                goodsite_Ch = goodchannels + '_channels'
                mychannel = globals()[goodsite_Ch]
                mychannelname = goodchannels
#                 print('mychannelname: '+ mychannelname)
#                 print(mychannel)

                result_map = {}

                for subcondition in myConditionDict[allCond]:
                    filenameNew=filenameBase+subcondition+'.csv'
                    file_exists = exists(filenameNew)
#                     print(filenameNew+str(file_exists))
                    if file_exists: 
                        res = Result(*filenameNew[:-4].rsplit('_', 1)) # why split?
#                         print(res)
                        raw_data = pd.read_csv(filenameNew, header=None)
                        cleaned_data = raw_data.rename(index=time_conversion_function(start_time, sample_interval), columns=convert_to_channel)
                        result_map[res] = cleaned_data
#                         for result in sorted(result_map.keys()):
#                         print(result)
                       
                        # we want to aggregate all waveforms of different conditions into one big variable (Waveform_All), and then update save_figure so that it loops over all Waveform_All, and plot them on the sasme figure with differnt colours and save it
                        waveform = average_channel_values(result_map[res], mychannel) #open waveforms for all channels for one participant, averaged
                        
                        maximal_event = find_maximum_and_mean_value_in_time_window(waveform, P1_start_time, P1_end_time)
                        print(res.subject+'|'+subcondition+'|'+mychannelname+'|'+'P1|'+"{:.4f}".format(maximal_event.value)+'|'+"{:.0f}".format(maximal_event.timestamp))
                        
# title_template = '{subject}|{condition}|{channel_name}|{ERP_peak}|{mean_amp}|{event_time}|{event_value}|{Status}'
#     title = title_template.format(
#         subject=experiment.subject,
#         condition=experiment.condition,
#         channel_name=channel_name,
#         ERP_peak=ERP,
#         mean_amp="{:.4f}".format(event.mean),
#         event_time=event.timestamp,
#         event_value="{:.4f}".format(event.value),
#         Status=event.status,
        
                        Waveform_All[res]=waveform
                 #   if result_map != {}:
                        #save_figure_Both(Waveform_All, result_map.keys(), mychannelname,'P1P3', P1_start_time, P3_start_time, P1_end_time, P3_end_time, allCond)
                        #save_figure_Both(Waveform_All, result_map.keys(), mychannelname,'LPP', LPP_start_time, LPP_start_time, LPP_end_time, LPP_end_time, allCond)

#del 'clear result_map'

# main = {'Oz':['Pz','Cz','PCz','POz'],'O1':['POL'],'O2':['POR'],'T5':['PL','CL'],'T6':['PR','CR']}
# channel_association_P3 = {'Oz':['Pz','Cz','PCz'],'T5':['PL','CL'],'T6':['PR','CR']}
# channel_association_Both = {'Oz':['POz','Oz'],'O1':['POL'],'O2':['POR'],'T5':['T5'],'T6':['T6']}



C:\QC\AD_08302011|Sad|POL|P1|7.6764|176
C:\QC\AD_08302011|Neu|POL|P1|9.8558|176
C:\QC\AD_08302011|Standard|POL|P1|7.3006|104
C:\QC\AD_08302011|Fear|POL|P1|10.6921|168
C:\QC\AD_08302011|Sad|POR|P1|14.5340|172
C:\QC\AD_08302011|Neu|POR|P1|14.2720|176
C:\QC\AD_08302011|Standard|POR|P1|11.1461|116
C:\QC\AD_08302011|Fear|POR|P1|13.0713|156
C:\QC\AD_08302011|Sad|POz|P1|12.5390|140
C:\QC\AD_08302011|Neu|POz|P1|14.0517|100
C:\QC\AD_08302011|Standard|POz|P1|9.6236|116
C:\QC\AD_08302011|Fear|POz|P1|12.8063|160
C:\QC\AD_08302011|Sad|Oz|P1|14.1997|172
C:\QC\AD_08302011|Neu|Oz|P1|13.8487|160
C:\QC\AD_08302011|Standard|Oz|P1|11.0863|116
C:\QC\AD_08302011|Fear|Oz|P1|13.6837|160
C:\QC\AD_08302011|Sad|T6|P1|9.2479|176
C:\QC\AD_08302011|Neu|T6|P1|9.6766|188
C:\QC\AD_08302011|Standard|T6|P1|6.3610|120
C:\QC\AD_08302011|Fear|T6|P1|6.8376|196
C:\QC\AL_07232011|Sad|POL|P1|11.9784|188
C:\QC\AL_07232011|Neu|POL|P1|12.4833|176
C:\QC\AL_07232011|Standard|POL|P1|6.7018|136
C:\QC\AL_07232011|Fear|POL|P1|7.1100|10

C:\QC\CD_08302011|Sad|Oz|P1|19.2663|128
C:\QC\CD_08302011|Neu|Oz|P1|27.0857|184
C:\QC\CD_08302011|Standard|Oz|P1|18.8420|120
C:\QC\CD_08302011|Fear|Oz|P1|17.5277|188
C:\QC\CD_08302011|Sad|T5|P1|7.7970|124
C:\QC\CD_08302011|Neu|T5|P1|21.0807|192
C:\QC\CD_08302011|Standard|T5|P1|9.0552|112
C:\QC\CD_08302011|Fear|T5|P1|7.7985|132
C:\QC\CD_08302011|Sad|T6|P1|12.7071|172
C:\QC\CD_08302011|Neu|T6|P1|22.7537|200
C:\QC\CD_08302011|Standard|T6|P1|9.9249|108
C:\QC\CD_08302011|Fear|T6|P1|12.4547|184
C:\QC\CE_10032012|Sad|POL|P1|33.6580|116
C:\QC\CE_10032012|Neu|POL|P1|30.6533|136
C:\QC\CE_10032012|Standard|POL|P1|28.8767|120
C:\QC\CE_10032012|Fear|POL|P1|27.2700|112
C:\QC\CE_10032012|Sad|POR|P1|38.3323|112
C:\QC\CE_10032012|Neu|POR|P1|36.1617|140
C:\QC\CE_10032012|Standard|POR|P1|31.2740|128
C:\QC\CE_10032012|Fear|POR|P1|32.6307|148
C:\QC\CE_10032012|Sad|POz|P1|39.0873|108
C:\QC\CE_10032012|Neu|POz|P1|34.4033|144
C:\QC\CE_10032012|Standard|POz|P1|31.3793|120
C:\QC\CE_10032012|Fear|POz|P1|28.5237|

C:\QC\JE_10052011|Sad|T5|P1|3.1710|160
C:\QC\JE_10052011|Neu|T5|P1|2.4607|172
C:\QC\JE_10052011|Standard|T5|P1|3.1313|156
C:\QC\JE_10052011|Fear|T5|P1|2.3781|196
C:\QC\JE_10052011|Sad|T6|P1|3.8254|96
C:\QC\JE_10052011|Neu|T6|P1|3.1188|176
C:\QC\JE_10052011|Standard|T6|P1|2.9633|156
C:\QC\JE_10052011|Fear|T6|P1|6.0130|156
C:\QC\JG_09262011|Sad|POL|P1|15.0317|92
C:\QC\JG_09262011|Neu|POL|P1|13.9863|100
C:\QC\JG_09262011|Standard|POL|P1|12.6944|100
C:\QC\JG_09262011|Fear|POL|P1|13.5447|96
C:\QC\JG_09262011|Sad|POR|P1|13.6764|92
C:\QC\JG_09262011|Neu|POR|P1|14.0495|100
C:\QC\JG_09262011|Standard|POR|P1|12.9977|100
C:\QC\JG_09262011|Fear|POR|P1|13.2376|148
C:\QC\JG_09262011|Sad|POz|P1|13.3677|88
C:\QC\JG_09262011|Neu|POz|P1|12.6193|96
C:\QC\JG_09262011|Standard|POz|P1|12.3470|96
C:\QC\JG_09262011|Fear|POz|P1|12.7320|140
C:\QC\JG_09262011|Sad|Oz|P1|17.4130|92
C:\QC\JG_09262011|Neu|Oz|P1|16.8780|104
C:\QC\JG_09262011|Standard|Oz|P1|15.6273|100
C:\QC\JG_09262011|Fear|Oz|P1|17.6543|100
C:\QC\JG

C:\QC\MH_09272011|Standard|POR|P1|15.6113|188
C:\QC\MH_09272011|Fear|POR|P1|10.3369|108
C:\QC\MH_09272011|Sad|POz|P1|17.5123|176
C:\QC\MH_09272011|Neu|POz|P1|11.6300|100
C:\QC\MH_09272011|Standard|POz|P1|14.3867|184
C:\QC\MH_09272011|Fear|POz|P1|10.0601|196
C:\QC\MH_09272011|Sad|Oz|P1|18.6453|176
C:\QC\MH_09272011|Neu|Oz|P1|12.6197|100
C:\QC\MH_09272011|Standard|Oz|P1|15.2177|180
C:\QC\MH_09272011|Fear|Oz|P1|10.8706|200
C:\QC\MH_09272011|Sad|T5|P1|9.6897|176
C:\QC\MH_09272011|Neu|T5|P1|9.3163|116
C:\QC\MH_09272011|Standard|T5|P1|7.8649|120
C:\QC\MH_09272011|Fear|T5|P1|10.8112|200
C:\QC\MH_09272011|Sad|T6|P1|8.0384|112
C:\QC\MH_09272011|Neu|T6|P1|6.3409|104
C:\QC\MH_09272011|Standard|T6|P1|5.5835|100
C:\QC\MH_09272011|Fear|T6|P1|4.5461|112
C:\QC\MJJ_05252013|Sad|POL|P1|15.8867|108
C:\QC\MJJ_05252013|Neu|POL|P1|16.8243|112
C:\QC\MJJ_05252013|Standard|POL|P1|16.9937|112
C:\QC\MJJ_05252013|Fear|POL|P1|17.3713|108
C:\QC\MJJ_05252013|Sad|POR|P1|16.1513|108
C:\QC\MJJ_05252013|Neu|POR|P1|16.48

C:\QC\RL_02062012|Standard|POz|P1|10.6517|88
C:\QC\RL_02062012|Fear|POz|P1|16.9077|96
C:\QC\RL_02062012|Sad|Oz|P1|12.4027|100
C:\QC\RL_02062012|Neu|Oz|P1|12.6913|100
C:\QC\RL_02062012|Standard|Oz|P1|11.8257|96
C:\QC\RL_02062012|Fear|Oz|P1|17.4080|148
C:\QC\RL_02062012|Sad|T5|P1|12.5287|116
C:\QC\RL_02062012|Neu|T5|P1|999.0000|999
C:\QC\RL_02062012|Standard|T5|P1|5.9216|96
C:\QC\RL_02062012|Fear|T5|P1|9.2137|112
C:\QC\RL_02062012|Sad|T6|P1|14.5187|112
C:\QC\RL_02062012|Neu|T6|P1|16.1477|116
C:\QC\RL_02062012|Standard|T6|P1|9.8087|124
C:\QC\RL_02062012|Fear|T6|P1|17.6247|124
C:\QC\RM_07182013|Sad|POL|P1|6.7941|200
C:\QC\RM_07182013|Neu|POL|P1|7.0444|176
C:\QC\RM_07182013|Standard|POL|P1|6.0999|92
C:\QC\RM_07182013|Fear|POL|P1|6.0574|176
C:\QC\RM_07182013|Sad|POR|P1|9.7865|92
C:\QC\RM_07182013|Neu|POR|P1|10.5939|176
C:\QC\RM_07182013|Standard|POR|P1|9.1950|92
C:\QC\RM_07182013|Fear|POR|P1|9.0651|180
C:\QC\RM_07182013|Sad|POz|P1|6.8696|200
C:\QC\RM_07182013|Neu|POz|P1|6.7689|176
C:\QC\RM_0

...

C:\QC\BS_02022011|TargetafterNeu|POR|P1|7.4601|104
C:\QC\BS_02022011|TargetafterTarget|POz|P1|7.9194|88
C:\QC\BS_02022011|TargetafterSad|POz|P1|14.8163|88
C:\QC\BS_02022011|TargetafterFear|POz|P1|13.7043|88
C:\QC\BS_02022011|TargetafterNeu|POz|P1|11.4093|92
C:\QC\BS_02022011|TargetafterTarget|Oz|P1|8.5451|88
C:\QC\BS_02022011|TargetafterSad|Oz|P1|16.0507|88
C:\QC\BS_02022011|TargetafterFear|Oz|P1|13.3657|88
C:\QC\BS_02022011|TargetafterNeu|Oz|P1|11.7248|96
C:\QC\BS_02022011|TargetafterTarget|T5|P1|2.5401|108
C:\QC\BS_02022011|TargetafterSad|T5|P1|9.3565|100
C:\QC\BS_02022011|TargetafterFear|T5|P1|6.8829|120
C:\QC\BS_02022011|TargetafterNeu|T5|P1|6.6690|108
C:\QC\BS_02022011|TargetafterTarget|T6|P1|2.8735|96
C:\QC\BS_02022011|TargetafterSad|T6|P1|7.2728|92
C:\QC\BS_02022011|TargetafterFear|T6|P1|5.9742|96
C:\QC\BS_02022011|TargetafterNeu|T6|P1|4.5464|108
C:\QC\CB_09302011|TargetafterTarget|POR|P1|12.2330|156
C:\QC\CB_09302011|TargetafterSad|POR|P1|17.8707|128
C:\QC\CB_09302011|Targetaft

C:\QC\ESP_12032011|TargetafterNeu|T5|P1|5.4642|96
C:\QC\ESP_12032011|TargetafterTarget|T6|P1|5.0643|92
C:\QC\ESP_12032011|TargetafterSad|T6|P1|2.6835|76
C:\QC\ESP_12032011|TargetafterFear|T6|P1|4.7537|88
C:\QC\ESP_12032011|TargetafterNeu|T6|P1|2.6085|72
C:\QC\HFG_02082011|TargetafterTarget|POL|P1|10.4653|68
C:\QC\HFG_02082011|TargetafterSad|POL|P1|-2.0573|188
C:\QC\HFG_02082011|TargetafterFear|POL|P1|10.1491|60
C:\QC\HFG_02082011|TargetafterNeu|POL|P1|10.2130|180
C:\QC\HFG_02082011|TargetafterTarget|POR|P1|12.9523|180
C:\QC\HFG_02082011|TargetafterSad|POR|P1|5.8464|192
C:\QC\HFG_02082011|TargetafterFear|POR|P1|16.0567|176
C:\QC\HFG_02082011|TargetafterNeu|POR|P1|13.3757|180
C:\QC\HFG_02082011|TargetafterTarget|POz|P1|11.5363|184
C:\QC\HFG_02082011|TargetafterSad|POz|P1|2.5648|192
C:\QC\HFG_02082011|TargetafterFear|POz|P1|10.7993|176
C:\QC\HFG_02082011|TargetafterNeu|POz|P1|11.9983|180
C:\QC\HFG_02082011|TargetafterTarget|Oz|P1|14.9873|64
C:\QC\HFG_02082011|TargetafterSad|Oz|P1|3.3547|1

C:\QC\KBB_09282012|TargetafterNeu|Oz|P1|8.3338|100
C:\QC\KBB_09282012|TargetafterTarget|T5|P1|6.0237|92
C:\QC\KBB_09282012|TargetafterSad|T5|P1|4.1739|180
C:\QC\KBB_09282012|TargetafterFear|T5|P1|1.6998|92
C:\QC\KBB_09282012|TargetafterNeu|T5|P1|5.5042|88
C:\QC\KBB_09282012|TargetafterTarget|T6|P1|2.5724|96
C:\QC\KBB_09282012|TargetafterSad|T6|P1|1.5167|104
C:\QC\KBB_09282012|TargetafterFear|T6|P1|1.9140|104
C:\QC\KBB_09282012|TargetafterNeu|T6|P1|4.3509|108
C:\QC\KF_10032012|TargetafterTarget|POL|P1|3.8402|116
C:\QC\KF_10032012|TargetafterSad|POL|P1|8.2948|124
C:\QC\KF_10032012|TargetafterFear|POL|P1|5.2803|156
C:\QC\KF_10032012|TargetafterNeu|POL|P1|8.4411|124
C:\QC\KF_10032012|TargetafterTarget|POR|P1|3.1733|116
C:\QC\KF_10032012|TargetafterSad|POR|P1|4.2505|172
C:\QC\KF_10032012|TargetafterFear|POR|P1|4.7776|156
C:\QC\KF_10032012|TargetafterNeu|POR|P1|8.3374|124
C:\QC\KF_10032012|TargetafterTarget|POz|P1|4.2553|116
C:\QC\KF_10032012|TargetafterSad|POz|P1|5.3609|128
C:\QC\KF_1003201

C:\QC\NJ_09262012|TargetafterNeu|T6|P1|9.4532|176
C:\QC\OC_04212012|TargetafterTarget|POz|P1|2.2414|76
C:\QC\OC_04212012|TargetafterSad|POz|P1|5.0279|196
C:\QC\OC_04212012|TargetafterFear|POz|P1|3.3544|84
C:\QC\OC_04212012|TargetafterNeu|POz|P1|5.5765|196
C:\QC\OC_04212012|TargetafterTarget|Oz|P1|2.3076|80
C:\QC\OC_04212012|TargetafterSad|Oz|P1|4.6846|196
C:\QC\OC_04212012|TargetafterFear|Oz|P1|3.4760|80
C:\QC\OC_04212012|TargetafterNeu|Oz|P1|5.5772|196
C:\QC\OC_04212012|TargetafterTarget|T5|P1|0.1929|68
C:\QC\OC_04212012|TargetafterSad|T5|P1|0.3206|196
C:\QC\OC_04212012|TargetafterFear|T5|P1|1.6859|72
C:\QC\OC_04212012|TargetafterNeu|T5|P1|1.7953|64
C:\QC\RF_08102012|TargetafterTarget|POL|P1|5.9350|112
C:\QC\RF_08102012|TargetafterSad|POL|P1|4.6677|104
C:\QC\RF_08102012|TargetafterFear|POL|P1|10.0779|120
C:\QC\RF_08102012|TargetafterNeu|POL|P1|5.0615|120
C:\QC\RF_08102012|TargetafterTarget|POR|P1|11.2885|92
C:\QC\RF_08102012|TargetafterSad|POR|P1|5.3614|100
C:\QC\RF_08102012|Targetaft

C:\QC\TL_08172012|TargetafterNeu|POL|P1|4.0690|108
C:\QC\TL_08172012|TargetafterTarget|POR|P1|6.6505|104
C:\QC\TL_08172012|TargetafterSad|POR|P1|6.4768|120
C:\QC\TL_08172012|TargetafterFear|POR|P1|9.9877|108
C:\QC\TL_08172012|TargetafterNeu|POR|P1|7.5855|104
C:\QC\TL_08172012|TargetafterTarget|POz|P1|7.2247|100
C:\QC\TL_08172012|TargetafterSad|POz|P1|4.9211|108
C:\QC\TL_08172012|TargetafterFear|POz|P1|8.9674|100
C:\QC\TL_08172012|TargetafterNeu|POz|P1|6.4805|104
C:\QC\TL_08172012|TargetafterTarget|Oz|P1|7.7924|104
C:\QC\TL_08172012|TargetafterSad|Oz|P1|7.1736|104
C:\QC\TL_08172012|TargetafterFear|Oz|P1|9.4956|100
C:\QC\TL_08172012|TargetafterNeu|Oz|P1|7.3365|104
C:\QC\TL_08172012|TargetafterTarget|T5|P1|1.6976|92
C:\QC\TL_08172012|TargetafterSad|T5|P1|4.3765|120
C:\QC\TL_08172012|TargetafterFear|T5|P1|6.0662|100
C:\QC\TL_08172012|TargetafterNeu|T5|P1|3.9065|108
C:\QC\TL_08172012|TargetafterTarget|T6|P1|4.0533|120
C:\QC\TL_08172012|TargetafterSad|T6|P1|6.4771|128
C:\QC\TL_08172012|Targe

In [29]:
myConditionDict = dict(
     AllDistractors = {'Fear','Neu','Sad','Standard'},
     AllTargets = {'TargetafterFear','TargetafterNeu','TargetafterSad','TargetafterTarget'}
)

result_map = {}
Waveform_All = {}
for allCond in myConditionDict.keys():
    input_file_pattern = good_path+'*_'+allCond+'_*.png'
    for filename in glob(input_file_pattern):
#         print(filename)
        mylocation = filename.find(allCond)
        last_slash=(filename.rfind('\\'))
        participantid = filename[last_slash+1:mylocation-1]
        goodsite = filename[mylocation+len(allCond)+1:mylocation+len(allCond)+3]
        filenameBase = my_path + participantid + '_'
#         print('filenameBase: '+ filenameBase)
        if goodsite in channel_association_LPP.keys():
#             print('good')
        
            for goodchannels in channel_association_LPP[goodsite]:
#                 print('goodchannels: '+ goodchannels)
                goodsite_Ch = goodchannels + '_channels'
                mychannel = globals()[goodsite_Ch]
                mychannelname = goodchannels
#                 print('mychannelname: '+ mychannelname)
#                 print(mychannel)

                result_map = {}

                for subcondition in myConditionDict[allCond]:
                    filenameNew=filenameBase+subcondition+'.csv'
                    file_exists = exists(filenameNew)
#                     print(filenameNew+str(file_exists))
                    if file_exists: 
                        res = Result(*filenameNew[:-4].rsplit('_', 1)) # why split?
#                         print(res)
                        raw_data = pd.read_csv(filenameNew, header=None)
                        cleaned_data = raw_data.rename(index=time_conversion_function(start_time, sample_interval), columns=convert_to_channel)
                        result_map[res] = cleaned_data
#                         for result in sorted(result_map.keys()):
#                         print(result)
                       
                        # we want to aggregate all waveforms of different conditions into one big variable (Waveform_All), and then update save_figure so that it loops over all Waveform_All, and plot them on the sasme figure with differnt colours and save it
                        waveform = average_channel_values(result_map[res], mychannel) #open waveforms for all channels for one participant, averaged
                        
                        maximal_event = find_maximum_and_mean_value_in_time_window(waveform, LPP_start_time, LPP_end_time)
                        print(res.subject+'|'+subcondition+'|'+mychannelname+'|'+'LPP|'+"{:.4f}".format(maximal_event.mean)+'|'+"{:.0f}".format(maximal_event.timestamp))
                        
# title_template = '{subject}|{condition}|{channel_name}|{ERP_peak}|{mean_amp}|{event_time}|{event_value}|{Status}'
#     title = title_template.format(
#         subject=experiment.subject,
#         condition=experiment.condition,
#         channel_name=channel_name,
#         ERP_peak=ERP,
#         mean_amp="{:.4f}".format(event.mean),
#         event_time=event.timestamp,
#         event_value="{:.4f}".format(event.value),
#         Status=event.status,
        
                        Waveform_All[res]=waveform
                 #   if result_map != {}:
                        #save_figure_Both(Waveform_All, result_map.keys(), mychannelname,'P1P3', P1_start_time, P3_start_time, P1_end_time, P3_end_time, allCond)
                        #save_figure_Both(Waveform_All, result_map.keys(), mychannelname,'LPP', LPP_start_time, LPP_start_time, LPP_end_time, LPP_end_time, allCond)

#del 'clear result_map'

# main = {'Oz':['Pz','Cz','PCz','POz'],'O1':['POL'],'O2':['POR'],'T5':['PL','CL'],'T6':['PR','CR']}
# channel_association_P3 = {'Oz':['Pz','Cz','PCz'],'T5':['PL','CL'],'T6':['PR','CR']}
# channel_association_Both = {'Oz':['POz','Oz'],'O1':['POL'],'O2':['POR'],'T5':['T5'],'T6':['T6']}



C:\QC\AD_08302011|Sad|Pz|LPP|18.6805|856
C:\QC\AD_08302011|Neu|Pz|LPP|3.2756|684
C:\QC\AD_08302011|Standard|Pz|LPP|1.8428|792
C:\QC\AD_08302011|Fear|Pz|LPP|10.1849|696
C:\QC\AD_08302011|Sad|PR|LPP|8.6525|884
C:\QC\AD_08302011|Neu|PR|LPP|4.5032|684
C:\QC\AD_08302011|Standard|PR|LPP|1.9126|688
C:\QC\AD_08302011|Fear|PR|LPP|9.5160|680
C:\QC\AD_08302011|Sad|T6|LPP|6.5537|884
C:\QC\AD_08302011|Neu|T6|LPP|4.1107|684
C:\QC\AD_08302011|Standard|T6|LPP|2.6725|688
C:\QC\AD_08302011|Fear|T6|LPP|10.0743|680
C:\QC\AL_07232011|Sad|Pz|LPP|7.7600|668
C:\QC\AL_07232011|Neu|Pz|LPP|8.9838|704
C:\QC\AL_07232011|Standard|Pz|LPP|-0.6132|712
C:\QC\AL_07232011|Fear|Pz|LPP|16.9035|672
C:\QC\AL_07232011|Sad|PL|LPP|1.0969|776
C:\QC\AL_07232011|Neu|PL|LPP|2.8064|656
C:\QC\AL_07232011|Standard|PL|LPP|0.5331|800
C:\QC\AL_07232011|Fear|PL|LPP|3.8282|812
C:\QC\AL_07232011|Sad|T5|LPP|4.1844|776
C:\QC\AL_07232011|Neu|T5|LPP|5.7635|652
C:\QC\AL_07232011|Standard|T5|LPP|1.5151|800
C:\QC\AL_07232011|Fear|T5|LPP|4.5507|740

C:\QC\CM_08102011|Standard|T6|LPP|6.2937|940
C:\QC\CM_08102011|Fear|T6|LPP|19.9509|676
C:\QC\CS_10022012|Sad|PL|LPP|-1.0433|748
C:\QC\CS_10022012|Neu|PL|LPP|4.4127|736
C:\QC\CS_10022012|Standard|PL|LPP|0.8569|732
C:\QC\CS_10022012|Fear|PL|LPP|6.1365|888
C:\QC\CS_10022012|Sad|T5|LPP|-4.0268|752
C:\QC\CS_10022012|Neu|T5|LPP|3.5051|744
C:\QC\CS_10022012|Standard|T5|LPP|-0.7440|772
C:\QC\CS_10022012|Fear|T5|LPP|15.5568|880
C:\QC\DG_10052012|Sad|Pz|LPP|3.8063|884
C:\QC\DG_10052012|Neu|Pz|LPP|-1.7785|676
C:\QC\DG_10052012|Standard|Pz|LPP|-0.4653|772
C:\QC\DG_10052012|Fear|Pz|LPP|6.7495|656
C:\QC\DG_10052012|Sad|PR|LPP|5.0667|660
C:\QC\DG_10052012|Neu|PR|LPP|4.8865|672
C:\QC\DG_10052012|Standard|PR|LPP|3.7231|944
C:\QC\DG_10052012|Fear|PR|LPP|10.2510|652
C:\QC\DG_10052012|Sad|T6|LPP|4.4361|652
C:\QC\DG_10052012|Neu|T6|LPP|6.8548|672
C:\QC\DG_10052012|Standard|T6|LPP|3.2661|940
C:\QC\DG_10052012|Fear|T6|LPP|7.3750|744
C:\QC\EK_10032011|Sad|Pz|LPP|0.9455|948
C:\QC\EK_10032011|Neu|Pz|LPP|2.5659|

C:\QC\KBB_09282012|Standard|PR|LPP|-1.1106|864
C:\QC\KBB_09282012|Fear|PR|LPP|3.1049|664
C:\QC\KBB_09282012|Sad|T6|LPP|-5.6462|676
C:\QC\KBB_09282012|Neu|T6|LPP|1.9309|912
C:\QC\KBB_09282012|Standard|T6|LPP|-0.1327|868
C:\QC\KBB_09282012|Fear|T6|LPP|-0.6695|764
C:\QC\KF_10032012|Sad|Pz|LPP|-6.4259|652
C:\QC\KF_10032012|Neu|Pz|LPP|-1.7890|668
C:\QC\KF_10032012|Standard|Pz|LPP|0.5596|808
C:\QC\KF_10032012|Fear|Pz|LPP|6.0675|848
C:\QC\KF_10032012|Sad|PL|LPP|2.0204|668
C:\QC\KF_10032012|Neu|PL|LPP|1.5123|676
C:\QC\KF_10032012|Standard|PL|LPP|-0.9720|668
C:\QC\KF_10032012|Fear|PL|LPP|8.3782|688
C:\QC\KF_10032012|Sad|T5|LPP|4.6423|664
C:\QC\KF_10032012|Neu|T5|LPP|1.1084|688
C:\QC\KF_10032012|Standard|T5|LPP|-0.8767|668
C:\QC\KF_10032012|Fear|T5|LPP|9.6644|832
C:\QC\KF_10032012|Sad|PR|LPP|1.1147|664
C:\QC\KF_10032012|Neu|PR|LPP|-0.1278|652
C:\QC\KF_10032012|Standard|PR|LPP|0.6615|812
C:\QC\KF_10032012|Fear|PR|LPP|5.2644|720
C:\QC\KF_10032012|Sad|T6|LPP|1.7307|668
C:\QC\KF_10032012|Neu|T6|LPP|

C:\QC\RL_02062012|Standard|Pz|LPP|0.8754|656
C:\QC\RL_02062012|Fear|Pz|LPP|16.1677|808
C:\QC\RL_02062012|Sad|PL|LPP|13.5986|732
C:\QC\RL_02062012|Neu|PL|LPP|8.2885|684
C:\QC\RL_02062012|Standard|PL|LPP|2.7573|828
C:\QC\RL_02062012|Fear|PL|LPP|18.3111|948
C:\QC\RL_02062012|Sad|T5|LPP|16.5990|684
C:\QC\RL_02062012|Neu|T5|LPP|12.2907|688
C:\QC\RL_02062012|Standard|T5|LPP|4.7198|828
C:\QC\RL_02062012|Fear|T5|LPP|25.9058|864
C:\QC\RL_02062012|Sad|PR|LPP|4.5871|660
C:\QC\RL_02062012|Neu|PR|LPP|13.3432|672
C:\QC\RL_02062012|Standard|PR|LPP|3.3267|800
C:\QC\RL_02062012|Fear|PR|LPP|19.2869|824
C:\QC\RL_02062012|Sad|T6|LPP|0.7472|688
C:\QC\RL_02062012|Neu|T6|LPP|13.5724|656
C:\QC\RL_02062012|Standard|T6|LPP|5.9987|816
C:\QC\RL_02062012|Fear|T6|LPP|27.0315|820
C:\QC\RM_07182013|Sad|Pz|LPP|1.2404|672
C:\QC\RM_07182013|Neu|Pz|LPP|-1.7996|884
C:\QC\RM_07182013|Standard|Pz|LPP|0.0703|872
C:\QC\RM_07182013|Fear|Pz|LPP|1.9275|652
C:\QC\RM_07182013|Sad|PL|LPP|-0.4968|932
C:\QC\RM_07182013|Neu|PL|LPP|0.4

C:\QC\AD_08302011|TargetafterSad|T5|LPP|11.1297|652
C:\QC\AD_08302011|TargetafterFear|T5|LPP|-8.2407|784
C:\QC\AD_08302011|TargetafterNeu|T5|LPP|14.6790|696
C:\QC\AD_08302011|TargetafterTarget|PR|LPP|-1.9754|660
C:\QC\AD_08302011|TargetafterSad|PR|LPP|-13.3746|784
C:\QC\AD_08302011|TargetafterFear|PR|LPP|22.0848|796
C:\QC\AD_08302011|TargetafterNeu|PR|LPP|-8.3141|948
C:\QC\AD_08302011|TargetafterTarget|T6|LPP|-2.8250|852
C:\QC\AD_08302011|TargetafterSad|T6|LPP|-22.9899|784
C:\QC\AD_08302011|TargetafterFear|T6|LPP|33.0831|792
C:\QC\AD_08302011|TargetafterNeu|T6|LPP|-9.6790|948
C:\QC\AL_07232011|TargetafterTarget|Pz|LPP|14.1321|912
C:\QC\AL_07232011|TargetafterSad|Pz|LPP|0.4322|692
C:\QC\AL_07232011|TargetafterFear|Pz|LPP|-7.4148|668
C:\QC\AL_07232011|TargetafterNeu|Pz|LPP|6.4534|668
C:\QC\AL_07232011|TargetafterTarget|PL|LPP|-0.5631|708
C:\QC\AL_07232011|TargetafterSad|PL|LPP|6.5528|788
C:\QC\AL_07232011|TargetafterFear|PL|LPP|-8.2788|668
C:\QC\AL_07232011|TargetafterNeu|PL|LPP|-1.3969|

C:\QC\CM_08102011|TargetafterSad|Pz|LPP|-25.2468|668
C:\QC\CM_08102011|TargetafterFear|Pz|LPP|-3.3237|780
C:\QC\CM_08102011|TargetafterNeu|Pz|LPP|-12.6056|732
C:\QC\CM_08102011|TargetafterTarget|PL|LPP|-3.3079|804
C:\QC\CM_08102011|TargetafterSad|PL|LPP|-25.9663|940
C:\QC\CM_08102011|TargetafterFear|PL|LPP|-0.0597|748
C:\QC\CM_08102011|TargetafterNeu|PL|LPP|-8.3587|740
C:\QC\CM_08102011|TargetafterTarget|T5|LPP|-2.2267|804
C:\QC\CM_08102011|TargetafterSad|T5|LPP|-14.4727|940
C:\QC\CM_08102011|TargetafterFear|T5|LPP|10.0091|852
C:\QC\CM_08102011|TargetafterNeu|T5|LPP|-1.0637|760
C:\QC\CM_08102011|TargetafterTarget|PR|LPP|-1.8418|900
C:\QC\CM_08102011|TargetafterSad|PR|LPP|-14.4482|936
C:\QC\CM_08102011|TargetafterFear|PR|LPP|0.8700|944
C:\QC\CM_08102011|TargetafterNeu|PR|LPP|-6.0146|732
C:\QC\CM_08102011|TargetafterTarget|T6|LPP|5.4670|896
C:\QC\CM_08102011|TargetafterSad|T6|LPP|-5.2789|932
C:\QC\CM_08102011|TargetafterFear|T6|LPP|10.2829|940
C:\QC\CM_08102011|TargetafterNeu|T6|LPP|2.18

C:\QC\JR_08022013|TargetafterSad|T6|LPP|3.4006|880
C:\QC\JR_08022013|TargetafterFear|T6|LPP|4.5765|840
C:\QC\JR_08022013|TargetafterNeu|T6|LPP|5.0268|932
C:\QC\KBB_09282012|TargetafterTarget|Pz|LPP|-0.1285|732
C:\QC\KBB_09282012|TargetafterSad|Pz|LPP|3.4907|656
C:\QC\KBB_09282012|TargetafterFear|Pz|LPP|4.1403|700
C:\QC\KBB_09282012|TargetafterNeu|Pz|LPP|-16.1977|800
C:\QC\KBB_09282012|TargetafterTarget|PL|LPP|0.2002|816
C:\QC\KBB_09282012|TargetafterSad|PL|LPP|-1.0840|656
C:\QC\KBB_09282012|TargetafterFear|PL|LPP|-2.0247|664
C:\QC\KBB_09282012|TargetafterNeu|PL|LPP|-0.8115|808
C:\QC\KBB_09282012|TargetafterTarget|T5|LPP|-2.8702|812
C:\QC\KBB_09282012|TargetafterSad|T5|LPP|-1.3695|700
C:\QC\KBB_09282012|TargetafterFear|T5|LPP|-3.1406|660
C:\QC\KBB_09282012|TargetafterNeu|T5|LPP|-2.9763|688
C:\QC\KBB_09282012|TargetafterTarget|PR|LPP|2.6273|672
C:\QC\KBB_09282012|TargetafterSad|PR|LPP|-2.7637|880
C:\QC\KBB_09282012|TargetafterFear|PR|LPP|1.3758|672
C:\QC\KBB_09282012|TargetafterNeu|PR|LP

C:\QC\RF_08102012|TargetafterTarget|PR|LPP|1.0471|696
C:\QC\RF_08102012|TargetafterSad|PR|LPP|2.3256|732
C:\QC\RF_08102012|TargetafterFear|PR|LPP|5.4363|940
C:\QC\RF_08102012|TargetafterNeu|PR|LPP|3.4755|888
C:\QC\RF_08102012|TargetafterTarget|T6|LPP|-0.7751|764
C:\QC\RF_08102012|TargetafterSad|T6|LPP|0.5858|732
C:\QC\RF_08102012|TargetafterFear|T6|LPP|3.5875|736
C:\QC\RF_08102012|TargetafterNeu|T6|LPP|0.6455|888
C:\QC\RG_10052012|TargetafterTarget|Pz|LPP|1.1299|676
C:\QC\RG_10052012|TargetafterSad|Pz|LPP|0.2534|664
C:\QC\RG_10052012|TargetafterFear|Pz|LPP|-3.3226|840
C:\QC\RG_10052012|TargetafterNeu|Pz|LPP|2.1050|672
C:\QC\RG_10052012|TargetafterTarget|PR|LPP|6.2833|756
C:\QC\RG_10052012|TargetafterSad|PR|LPP|4.4113|868
C:\QC\RG_10052012|TargetafterFear|PR|LPP|3.0343|836
C:\QC\RG_10052012|TargetafterNeu|PR|LPP|4.8792|792
C:\QC\RG_10052012|TargetafterTarget|T6|LPP|5.6099|920
C:\QC\RG_10052012|TargetafterSad|T6|LPP|2.4148|864
C:\QC\RG_10052012|TargetafterFear|T6|LPP|3.7358|940
C:\QC\RG_